In [1]:
#| default_exp spice.kernels

# SPICE Kernels
> Tools to manage SPICE kernels

## Intro

Feature list for this module:

* Receive the list of supported datasets for automatic retrieval of archived SPICE kernels
  * The supported datasets are tabled here at NAIF: https://naif.jpl.nasa.gov/naif/data_archived.html
* Receive the list of required SPICE kernels for a given mission and time range
* Automatic download of kernels for a given mission and time range either into a given location or the `planetarypy` local archive.

As always in `planetarypy` the general design philosophy is to first develop a management class to give the user full control over all the details, and then add easy-to-use function for the end-user that do the most frequently used things in one go. (See section "User Functions")

In [2]:
#| export
import warnings
import zipfile
from datetime import timedelta
from io import BytesIO
from itertools import repeat
from multiprocessing import Pool, cpu_count
from pathlib import Path

import requests
import spiceypy as spice
from astropy.time import Time
from dask.distributed import Client
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from yarl import URL

import pandas as pd
from fastcore.test import test_fail
from fastcore.utils import store_attr
from planetarypy.config import config
from planetarypy.utils import nasa_time_to_iso, url_retrieve

In [3]:
#| hide
from nbdev.showdoc import show_doc

In [41]:
#| export
KERNEL_STORAGE = config.storage_root / "spice_kernels"
KERNEL_STORAGE.mkdir(exist_ok=True, parents=True)

## Identifying and downloading kernel sets

One repeating task for SPICE calculations is the identification and retrieval of all SPICE kernels for a mission for a given time interval.

The folks at NAIF offer a "Subset" feature at their servers.
Here we set up a table of the currently supported datasets:

In [42]:
#| export
dataset_ids = {
    "cassini": "co-s_j_e_v-spice-6-v1.0/cosp_1000",
    "clementine": "clem1-l-spice-6-v1.0/clsp_1000",
    "dawn": "dawn-m_a-spice-6-v1.0/dawnsp_1000",
    "di": "di-c-spice-6-v1.0/disp_1000",
    "ds1": "ds1-a_c-spice-6-v1.0/ds1sp_1000",
    "epoxi": "dif-c_e_x-spice-6-v1.0/epxsp_1000",
    "em16": "em16/em16_spice",
    "grail": "grail-l-spice-6-v1.0/grlsp_1000",
    "hayabusa": "hay-a-spice-6-v1.0/haysp_1000",
    "insight": "insight/insight_spice",
    "juno": "jno-j_e_ss-spice-6-v1.0/jnosp_1000",
    "ladee": "ladee/ladee_spice",
    "lro": "lro-l-spice-6-v1.0/lrosp_1000",
    "maven": "maven/maven_spice",
    "opportunity": "mer1-m-spice-6-v1.0/mer1sp_1000",
    "spirit": "mer2-m-spice-6-v1.0/mer2sp_1000",
    "messenger": "mess-e_v_h-spice-6-v1.0/messsp_1000",
    "mars2020": "mars2020/mars2020_spice",
    "mex": "mex-e_m-spice-6-v2.0/mexsp_2000",
    "mgs": "mgs-m-spice-6-v1.0/mgsp_1000",
    "ody": "ody-m-spice-6-v1.0/odsp_1000",
    "mro": "mro-m-spice-6-v1.0/mrosp_1000",
    "msl": "msl-m-spice-6-v1.0/mslsp_1000",
    "near": "near-a-spice-6-v1.0/nearsp_1000",
    "nh": "nh-j_p_ss-spice-6-v1.0/nhsp_1000",
    "orex": "orex/orex_spice",
    "rosetta": "ro_rl-e_m_a_c-spice-6-v1.0/rossp_1000",
    "stardust": "sdu-c-spice-6-v1.0/sdsp_1000",
    "venus_climate_orbiter": "vco/vco_spice",
    "vex": "vex-e_v-spice-6-v2.0/vexsp_2000",
    "vo": "vo1_vo2-m-spice-6-v1.0/vosp_1000",
}

df = pd.DataFrame({"shorthand": dataset_ids.keys(), "path": dataset_ids.values()})

df2 = pd.read_html("https://naif.jpl.nasa.gov/naif/data_archived.html")[6]
df2.columns = df2.iloc[0]
df2 = df2.drop(0).reset_index(drop=True)
df2 = df2.drop(["Archive Readme", "Archive Link", "Subset Link"], axis=1)
df = df.join(df2)
datasets = df.set_index("shorthand")

In [43]:
datasets

,path,Mission Name,PDS3 or PDS4,Data Size (GB),Start Time,Stop Time
shorthand,,,,,,
cassini,co-s_j_e_v-spice-6-v1.0/cosp_1000,Cassini Orbiter,3,62.5,1997-10-15,2017-09-15
clementine,clem1-l-spice-6-v1.0/clsp_1000,Clementine,3,0.8,1994-01-26,1994-05-07
dawn,dawn-m_a-spice-6-v1.0/dawnsp_1000,DART,4,9.1,2021-11-09,2050-01-01
di,di-c-spice-6-v1.0/disp_1000,DAWN,3,86.4,2007-09-27,2018-10-31
ds1,ds1-a_c-spice-6-v1.0/ds1sp_1000,Deep Impact,3,0.7,2005-01-12,2005-08-09
epoxi,dif-c_e_x-spice-6-v1.0/epxsp_1000,Deep Space 1,3,0.9,1998-10-24,2001-12-18
em16,em16/em16_spice,EPOXI,3,1.0,2005-08-23,2011-03-01
grail,grail-l-spice-6-v1.0/grlsp_1000,ExoMars TGO 2016,4,9.4,2016-03-14,2023-01-01
hayabusa,hay-a-spice-6-v1.0/haysp_1000,GRAIL,3,4.3,2011-09-10,2012-12-17


To receive this dataframe:

```python
from planetarypy.spice.kernels import datasets
```
Some validation helpers:

In [44]:
#| export
def is_start_valid(
    mission: str,  # mission shorthand label of datasets dataframe
    start: Time,  # start time in astropy.Time format
):
    return Time(datasets.at[mission, "Start Time"]) <= start


def is_stop_valid(
    mission: str,  # mission shorthand label of datasets dataframe
    stop: Time,  # stop time in astropy.Time format
):
    return Time(datasets.at[mission, "Stop Time"]) >= stop

In [45]:
assert is_start_valid("cassini", Time("1998-01-01")) is True
assert is_start_valid("cassini", Time("1997-01-01")) is False
assert is_stop_valid("cassini", "2017-01-01") is True
assert is_stop_valid("cassini", "2018-01-01") is False

Now we build a management class for wrapping the Perl script available at below's URL for accessing subsets of these datasets.

First, the basic URLs we will use:

In [46]:
#| export
NAIF_URL = URL("https://naif.jpl.nasa.gov")
BASE_URL = NAIF_URL / "cgi-bin/subsetds.pl"

The Perl script `subsetds.pl` (the name at the end of the `BASE_URL`) requires as input:

* the dataset name
* start and stop of the time interval
* a constant named "Subset" to identify the action for this Perl script

We can assemble these parameters into a payload dictionary for the `requests.get` call and we manage different potential actions on the zipfile with a `Subsetter` class, that only requires the mission identifier, start and stop as parameters.

In [47]:
#| export

def download_one_url(url, local_path, overwrite: bool = False):
    if local_path.exists() and not overwrite:
        return
    local_path.parent.mkdir(exist_ok=True, parents=True)
    url_retrieve(url, local_path)

class Subsetter:
    """Class to manage retrieving subset SPICE kernel lists

    Attributes
    ----------
    kernel_names: List of
    """

    def __init__(
        self,
        mission: str,  # mission shorthand in datasets dataframe
        start: str,  # start time in either ISO or yyyy-jjj format
        stop=None,  # stop time in either ISO or yyyy-jjj format
        save_location=None,  # overwrite default storing in planetarpy archive
    ):
        store_attr()
        self.initialize()

    def initialize(self):
        r = self.r
        if r.ok:
            z = zipfile.ZipFile(BytesIO(r.content))
        else:
            raise IOError("SPICE Server request returned status code: {r.status_code}")
        self.z = z
        # these files only exist "virtually" in the zip object, but are needed to
        # extract them:
        self.urls_file = [n for n in z.namelist() if n.startswith("urls_")][0]
        self.metakernel_file = [n for n in z.namelist() if n.lower().endswith(".tm")][0]
        with self.z.open(self.urls_file) as f:
            self.kernel_urls = f.read().decode().split()

    @property
    def r(self):
        return requests.get(BASE_URL, params=self.payload, stream=True)

    @property
    def start(self):
        return self._start

    @start.setter
    def start(self, value):
        try:
            self._start = Time(value)
        except ValueError:
            self._start = Time(nasa_time_to_iso(value))

    @property
    def stop(self):
        return self._stop

    @stop.setter
    def stop(self, value):
        if not value:
            self._stop = self.start + timedelta(days=1)
        else:
            try:
                self._stop = Time(value)
            except ValueError:
                self._stop = Time(nasa_time_to_iso(value))

    @property
    def payload(self):
        """Put payload together while checking for working time format.

        If Time(self.start) doesn't work, then we assume that the date must be in the
        Time-unsupported yyyy-jjj format, which can be converted by `nasa_time_to_iso`
        from `planetarypy.utils`.
        """
        if not (
            is_start_valid(self.mission, self.start)
            and is_stop_valid(self.mission, self.stop)
        ):
            raise ValueError(
                "One of start/stop is outside the supported date-range. See `datasets`."
            )
        p = {
            "dataset": dataset_ids[self.mission],
            "start": self.start.iso,
            "stop": self.stop.iso,
            "action": "Subset",
        }
        return p

    @property
    def kernel_names(self):
        "Return list of names of kernels for the given time range."
        return [
            str(Path(URL(url).parent.name) / URL(url).name) for url in self.kernel_urls
        ]

    def get_local_path(
        self,
        url,  # kernel url to determine local storage path
    ) -> Path:  # full local path where kernel in URL will be stored
        """Calculate local storage path from Kernel URL, using `save_location` if given.

        If self.save_location is None, the `planetarypy` archive is being used.
        """
        u = URL(url)
        basepath = (
            KERNEL_STORAGE / self.mission
            if not self.save_location
            else self.save_location
        )
        return basepath / u.parent.name / u.name

    def _non_blocking_download(self, overwrite: bool = False):
        with Client() as client:
            futures = []
            for url in tqdm(self.kernel_urls, desc="Kernels downloaded"):
                local_path = self.get_local_path(url)
                if local_path.exists() and not overwrite:
                    print(local_path.parent.name, local_path.name, "locally available.")
                    continue
                local_path.parent.mkdir(exist_ok=True, parents=True)
                futures.append(client.submit(url_retrieve, url, local_path))
            return [f.result() for f in futures]
        
    def _concurrent_download(self, overwrite: bool = False):
        paths = [self.get_local_path(url) for url in self.kernel_urls]
        args = zip(self.kernel_urls, paths, repeat(overwrite))
        results = process_map(download_one_url, args, max_workers=cpu_count()-2)

    def download_kernels(
        self,
        overwrite: bool = False,  # switch to control if kernels should be downloaded over existing ones
        non_blocking: bool = False,
    ):
        if non_blocking:
            return self._non_blocking_download(overwrite)
        # sequential download
        for url in tqdm(self.kernel_urls, desc="Kernels downloaded"):
            local_path = self.get_local_path(url)
            if local_path.exists() and not overwrite:
                print(local_path.parent.name, local_path.name, "locally available.")
                continue
            local_path.parent.mkdir(exist_ok=True, parents=True)
            url_retrieve(url, local_path)

    def get_metakernel(self) -> Path:  # return path to metakernel file
        """Get metakernel file from NAIF and adapt path to match local storage.

        Use `save_location` if given, otherwise `planetarypy` archive.
        """
        basepath = (
            KERNEL_STORAGE / self.mission
            if not self.save_location
            else self.save_location
        )
        savepath = basepath / self.metakernel_file
        with open(savepath, "w") as outfile, self.z.open(
            self.metakernel_file
        ) as infile:
            for line in infile:
                linestr = line.decode()
                if "'./data'" in linestr:
                    linestr = linestr.replace("'./data'", f"'{savepath.parent}'")
                outfile.write(linestr)
        return savepath

In [48]:
subset = Subsetter("cassini", "2014-270")

In [49]:
subset.kernel_names

['ck/14001_15001pa_gapfill_v14.bc',
 'ck/14212_14279py_as_flown.bc',
 'ck/14268_14273ra.bc',
 'ck/cas_cda_20150318.bc',
 'ck/cas_lemms_05109_20001_v2.bc',
 'fk/cas_dyn_v03.tf',
 'fk/cas_mimi_v202.tf',
 'fk/cas_rocks_v18.tf',
 'fk/cas_v41.tf',
 'ik/cas_caps_v03.ti',
 'ik/cas_cda_v01.ti',
 'ik/cas_cirs_v09.ti',
 'ik/cas_inms_v02.ti',
 'ik/cas_iss_v10.ti',
 'ik/cas_mag_v01.ti',
 'ik/cas_mimi_v11.ti',
 'ik/cas_radar_v11.ti',
 'ik/cas_rpws_v01.ti',
 'ik/cas_rss_v03.ti',
 'ik/cas_sru_v02.ti',
 'ik/cas_uvis_v06.ti',
 'ik/cas_vims_v06.ti',
 'lsk/naif0012.tls',
 'pck/pck00010.tpc',
 'sclk/cas00172.tsc',
 'spk/140809BP_IRRE_00256_25017.bsp',
 'spk/150122R_SCPSE_14251_14283.bsp',
 'spk/180927AP_RE_90165_18018.bsp']

In [50]:
# this should fail:
def _failing():
    Subsetter("cassini", "2019-01-01")


test_fail(_failing, contains="start/stop")

In [51]:
subset = Subsetter("cassini", "2011-02-13", "2011-02-14")

In [52]:
subset.urls_file

'urls_cosp_1000_110213_110214.txt'

In [53]:
subset.metakernel_file

'cas_2011_v17_110213_110214.tm'

In [54]:
show_doc(Subsetter.kernel_names)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L190){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.kernel_names

>      Subsetter.kernel_names ()

Return list of names of kernels for the given time range.

In [55]:
subset.kernel_names

['ck/11001_12001pa_gapfill_v14.bc',
 'ck/11017_11066py_as_flown.bc',
 'ck/11044_11049ra.bc',
 'ck/cas_cda_20120517.bc',
 'ck/cas_lemms_05109_20001_v2.bc',
 'fk/cas_dyn_v03.tf',
 'fk/cas_mimi_v202.tf',
 'fk/cas_rocks_v18.tf',
 'fk/cas_v41.tf',
 'ik/cas_caps_v03.ti',
 'ik/cas_cda_v01.ti',
 'ik/cas_cirs_v09.ti',
 'ik/cas_inms_v02.ti',
 'ik/cas_iss_v10.ti',
 'ik/cas_mag_v01.ti',
 'ik/cas_mimi_v11.ti',
 'ik/cas_radar_v11.ti',
 'ik/cas_rpws_v01.ti',
 'ik/cas_rss_v03.ti',
 'ik/cas_sru_v02.ti',
 'ik/cas_uvis_v06.ti',
 'ik/cas_vims_v06.ti',
 'lsk/naif0012.tls',
 'pck/pck00010.tpc',
 'sclk/cas00172.tsc',
 'spk/110504R_SCPSE_11041_11093.bsp',
 'spk/140809BP_IRRE_00256_25017.bsp',
 'spk/180927AP_RE_90165_18018.bsp']

In [56]:
subset.kernel_urls

['https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/11001_12001pa_gapfill_v14.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/11017_11066py_as_flown.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/11044_11049ra.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/cas_cda_20120517.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/ck/cas_lemms_05109_20001_v2.bc',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/fk/cas_dyn_v03.tf',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/fk/cas_mimi_v202.tf',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/fk/cas_rocks_v18.tf',
 'https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000/data/fk/cas_v41.tf',
 'https://

In [57]:
show_doc(Subsetter.get_local_path)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L196){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.get_local_path

>      Subsetter.get_local_path (url)

Calculate local storage path from Kernel URL, using `save_location` if given.

If self.save_location is None, the `planetarypy` archive is being used.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| url |  | kernel url to determine local storage path |
| **Returns** | **Path** | **full local path where kernel in URL will be stored** |

In [58]:
subset.get_local_path(subset.kernel_urls[0])

Path('/home/ayek72/mnt/slowdata/planetarypy/spice_kernels/cassini/ck/11001_12001pa_gapfill_v14.bc')

In [59]:
subset.save_location = Path(".")

In [60]:
subset.get_local_path(subset.kernel_urls[0])

Path('ck/11001_12001pa_gapfill_v14.bc')

In [61]:
show_doc(Subsetter.download_kernels)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L229){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.download_kernels

>      Subsetter.download_kernels (overwrite:bool=False,
>                                  non_blocking:bool=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| overwrite | bool | False | switch to control if kernels should be downloaded over existing ones |
| non_blocking | bool | False |  |

In [62]:
# reset save_location to prevent additional download
subset.save_location = None

In [63]:
subset.download_kernels()

Kernels downloaded:   0%|          | 0/28 [00:00<?, ?it/s]

ck 11001_12001pa_gapfill_v14.bc locally available.
ck 11017_11066py_as_flown.bc locally available.
ck 11044_11049ra.bc locally available.
ck cas_cda_20120517.bc locally available.
ck cas_lemms_05109_20001_v2.bc locally available.
fk cas_dyn_v03.tf locally available.
fk cas_mimi_v202.tf locally available.
fk cas_rocks_v18.tf locally available.
fk cas_v41.tf locally available.
ik cas_caps_v03.ti locally available.
ik cas_cda_v01.ti locally available.
ik cas_cirs_v09.ti locally available.
ik cas_inms_v02.ti locally available.
ik cas_iss_v10.ti locally available.
ik cas_mag_v01.ti locally available.
ik cas_mimi_v11.ti locally available.
ik cas_radar_v11.ti locally available.
ik cas_rpws_v01.ti locally available.
ik cas_rss_v03.ti locally available.
ik cas_sru_v02.ti locally available.
ik cas_uvis_v06.ti locally available.
ik cas_vims_v06.ti locally available.
lsk naif0012.tls locally available.
pck pck00010.tpc locally available.
sclk cas00172.tsc locally available.
spk 110504R_SCPSE_11041

In [64]:
subset._concurrent_download()

0it [00:00, ?it/s]

TypeError: download_one_url() missing 1 required positional argument: 'local_path'

In [ ]:
subset.download_kernels(non_blocking=True)

In [65]:
show_doc(Subsetter.get_metakernel)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/spice/kernels.py#L245){target="_blank" style="float:right; font-size:smaller"}

### Subsetter.get_metakernel

>      Subsetter.get_metakernel ()

Get metakernel file from NAIF and adapt path to match local storage.

Use `save_location` if given, otherwise `planetarypy` archive.

In [66]:
mkpath = subset.get_metakernel()
mkpath

Path('/home/ayek72/mnt/slowdata/planetarypy/spice_kernels/cassini/cas_2011_v17_110213_110214.tm')

In [67]:
!cat {mkpath}

overwriting variable {'ISISROOT', 'ISISDATA'}
KPL/MK

   This meta-kernel lists a subset of kernels from the meta-kernel
   cas_2011_v17.tm provided in the CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive,
   covering the whole or a part of the customer requested time period
   from 2011-02-13T00:00:00.000 to 2011-02-14T00:00:00.000.

   The documentation describing these kernels can be found in the
   complete CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive available at this URL

   https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000

   To use this meta-kernel users may need to modify the value of the
   PATH_VALUES keyword to point to the actual location of the archive's
   ``data'' directory on their system. Replacing ``/'' with ``\''
   and converting line terminators to the format native to the user's
   system may also be required if this meta-kernel is to be used on a
   non-UNIX workstation.

   This meta-kernel was created by the NAIF node's SPICE PDS archive
 

Loading the metakernel works! :

In [68]:
spice.furnsh(str(mkpath))

Or, with given `save_location`:

In [69]:
subset.save_location = Path(".")
mkpath = subset.get_metakernel()
mkpath

Path('cas_2011_v17_110213_110214.tm')

The metakernel is correctly adapted, however for these tests, I didn't download the kernels again

In [70]:
!cat {mkpath}

overwriting variable {'ISISROOT', 'ISISDATA'}
KPL/MK

   This meta-kernel lists a subset of kernels from the meta-kernel
   cas_2011_v17.tm provided in the CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive,
   covering the whole or a part of the customer requested time period
   from 2011-02-13T00:00:00.000 to 2011-02-14T00:00:00.000.

   The documentation describing these kernels can be found in the
   complete CO-S/J/E/V-SPICE-6-V1.0 SPICE PDS3 archive available at this URL

   https://naif.jpl.nasa.gov/pub/naif/pds/data/co-s_j_e_v-spice-6-v1.0/cosp_1000

   To use this meta-kernel users may need to modify the value of the
   PATH_VALUES keyword to point to the actual location of the archive's
   ``data'' directory on their system. Replacing ``/'' with ``\''
   and converting line terminators to the format native to the user's
   system may also be required if this meta-kernel is to be used on a
   non-UNIX workstation.

   This meta-kernel was created by the NAIF node's SPICE PDS archive
 

## User functions

In [71]:
#| export
def get_metakernel_and_files(
    mission: str,  # mission shorthand from datasets dataframe
    start: str,  # start time as iso-string, or yyyy-jjj
    stop: str,  # stop time as iso-string or yyyy-jjj
    save_location: str = None,  # override storage into planetarypy archive
) -> Path:  # pathlib.Path to metakernel file with corrected data path.
    "For a given mission and start/stop times, download the kernels and get metakernel path"
    subset = Subsetter(mission, start, stop, save_location)
    subset.download_kernels(non_blocking=True)
    return subset.get_metakernel()

In [72]:
mkpath = get_metakernel_and_files("cassini", "2011-02-13", "2011-02-14")
mkpath

Kernels downloaded:   0%|          | 0/28 [00:00<?, ?it/s]

ck 11001_12001pa_gapfill_v14.bc locally available.
ck 11017_11066py_as_flown.bc locally available.
ck 11044_11049ra.bc locally available.
ck cas_cda_20120517.bc locally available.
ck cas_lemms_05109_20001_v2.bc locally available.
fk cas_dyn_v03.tf locally available.
fk cas_mimi_v202.tf locally available.
fk cas_rocks_v18.tf locally available.
fk cas_v41.tf locally available.
ik cas_caps_v03.ti locally available.
ik cas_cda_v01.ti locally available.
ik cas_cirs_v09.ti locally available.
ik cas_inms_v02.ti locally available.
ik cas_iss_v10.ti locally available.
ik cas_mag_v01.ti locally available.
ik cas_mimi_v11.ti locally available.
ik cas_radar_v11.ti locally available.
ik cas_rpws_v01.ti locally available.
ik cas_rss_v03.ti locally available.
ik cas_sru_v02.ti locally available.
ik cas_uvis_v06.ti locally available.
ik cas_vims_v06.ti locally available.
lsk naif0012.tls locally available.
pck pck00010.tpc locally available.
sclk cas00172.tsc locally available.
spk 110504R_SCPSE_11041

Path('/home/ayek72/mnt/slowdata/planetarypy/spice_kernels/cassini/cas_2011_v17_110213_110214.tm')

In [73]:
#| export
def list_kernels_for_day(
    mission: str,  # mission shorthand from datasets dataframe
    start: str,  # start time as iso-string, or yyyy-jjj
    stop: str = "",  # stop time as iso-string or yyyy-jjj
) -> list:  # list of kernel names
    subset = Subsetter(mission, start, stop)
    return subset.kernel_names

In [74]:
list_kernels_for_day("mro", "2015-02-13")

['ck/mro_crm_psp_150201_150228.bc',
 'ck/mro_hga_psp_150210_150216_v2.bc',
 'ck/mro_mcs_psp_150201_150228.bc',
 'ck/mro_sa_psp_150210_150216_v2.bc',
 'ck/mro_sc_psp_150210_150216_v2.bc',
 'fk/mro_v16.tf',
 'ik/mro_crism_v10.ti',
 'ik/mro_ctx_v11.ti',
 'ik/mro_hirise_v12.ti',
 'ik/mro_marci_v10.ti',
 'ik/mro_mcs_v10.ti',
 'ik/mro_onc_v10.ti',
 'lsk/naif0012.tls',
 'pck/pck00008.tpc',
 'sclk/mro_sclkscet_00095_65536.tsc',
 'spk/de421.bsp',
 'spk/mar097.bsp',
 'spk/mro_psp34.bsp',
 'spk/mro_psp34_ssd_mro95a.bsp',
 'spk/mro_struct_v10.bsp']

In [75]:
list_kernels_for_day("maven", "2017-01-01")

['ck/mvn_app_rel_161226_170101_v01.bc',
 'ck/mvn_app_rel_170102_170108_v01.bc',
 'ck/mvn_iuvs_rem_170101_170331_v03.bc',
 'ck/mvn_sc_rel_161226_170101_v01.bc',
 'ck/mvn_sc_rel_170102_170108_v02.bc',
 'ck/mvn_swea_nom_131118_300101_v02.bc',
 'fk/maven_v09.tf',
 'ik/maven_ant_v10.ti',
 'ik/maven_euv_v10.ti',
 'ik/maven_iuvs_v11.ti',
 'ik/maven_ngims_v10.ti',
 'ik/maven_sep_v12.ti',
 'ik/maven_static_v11.ti',
 'ik/maven_swea_v11.ti',
 'ik/maven_swia_v10.ti',
 'lsk/naif0012.tls',
 'pck/pck00010.tpc',
 'sclk/mvn_sclkscet_00072.tsc',
 'spk/de430s.bsp',
 'spk/mar097s.bsp',
 'spk/maven_orb_rec_170101_170401_v1.bsp',
 'spk/maven_struct_v01.bsp']

In [76]:
# |filter_stream ErfaWarning


def _test_mission_kernels_available(mission):
    start = datasets.at[mission, "Start Time"]
    end = datasets.at[mission, "Stop Time"]
    half = Time(start) + (Time(end) - Time(start)) / 2
    print("Half time:", half)
    try:
        found = list_kernels_for_day(mission, half)
    except IndexError:
        print("Problem with", mission)
    else:
        print(f"Found {len(found)} kernels for {mission}")


futures = []
with Client() as client:
    for mission in datasets.index:
        futures.append(client.submit(_test_mission_kernels_available, mission))
    [f.result() for f in futures]

/home/ayek72/mambaforge/envs/py310/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/ayek72/mambaforge/envs/py310/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/ayek72/mambaforge/envs/py310/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/ayek72/mambaforge/envs/py310/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/ayek72/mambaforge/envs/py310/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarn

Half time: 2008-05-27 00:00:00.000
Half time: 2035-12-06 00:00:00.000
Half time: 2018-06-10 11:59:59.000
Found 6 kernels for em16
Found 21 kernels for opportunity
Half time: 2007-09-30 12:00:01.000
Half time: 2005-04-26 12:00:00.000
Found 9 kernels for ds1
Half time: 2031-11-04 11:59:59.000
Found 32 kernels for cassini
Half time: 2012-04-29 00:00:00.500
Found 10 kernels for hayabusa
Half time: 2013-04-14 00:00:00.000
Half time: 2019-08-07 23:59:59.500
Found 10 kernels for di
Half time: 2010-12-23 00:00:00.500
Found 16 kernels for grail
Half time: 2016-12-27 23:59:59.500
Found 20 kernels for spirit
Found 12 kernels for ladee
Half time: 1994-03-17 12:00:00.000
Half time: 2000-05-21 11:59:59.500
Found 19 kernels for clementine
Found 7 kernels for epoxi
Half time: 2016-01-31 11:59:59.500
Half time: 2005-10-15 12:00:00.000
Found 3 kernels for insight
Found 22 kernels for maven


BadZipFile: File is not a zip file

> NOTE: Any ErfaWarnings above are caused by the LADEE mission using a kernel up to 2050, and the astropy.Time module warns about potential precicision issues regarding unknown leapseconds that will be put in in the future.

### Generic kernel management

There are a few generic kernels that are required for basic illumination calculations as supported by this package.

In [85]:
#| export

GENERIC_URL = NAIF_URL / "pub/naif/generic_kernels/"

generic_kernel_names = [
    "lsk/naif0012.tls",
    "pck/pck00010.tpc",
    "pck/de-403-masses.tpc",
    "spk/planets/de430.bsp",
    "spk/satellites/mar097.bsp",
]
generic_kernel_paths = [GENERIC_STORAGE.joinpath(i) for i in generic_kernel_names]

In [86]:
#| export
GENERIC_STORAGE = KERNEL_STORAGE / "generic"
GENERIC_STORAGE.mkdir(exist_ok=True, parents=True)
generic_kernel_paths = [
    GENERIC_STORAGE.joinpath(i) for i in generic_kernel_names
]

In [87]:
#| export


def download_generic_kernels(overwrite=False):
    "Download all kernels as required by generic_kernel_list."
    dl_urls = [GENERIC_URL / i for i in generic_kernel_names]
    for dl_url, savepath in zip(dl_urls, generic_kernel_paths):
        if savepath.exists() and not overwrite:
            print(
                savepath.name,
                "already downloaded. Use `overwrite=True` to download again.")
            continue
        savepath.parent.mkdir(exist_ok=True, parents=True)
        url_retrieve(dl_url, savepath)

In [88]:
download_generic_kernels()

naif0012.tls already downloaded. Use `overwrite=True` to download again.
pck00010.tpc already downloaded. Use `overwrite=True` to download again.
de-403-masses.tpc already downloaded. Use `overwrite=True` to download again.
de430.bsp already downloaded. Use `overwrite=True` to download again.
mar097.bsp already downloaded. Use `overwrite=True` to download again.


In [111]:
#| export


def load_generic_kernels():
    """Load all kernels in generic_kernels list.

    Loads pure planetary bodies meta-kernel without spacecraft data.

    Downloads any missing generic kernels.
    """
    if any([not p.exists() for p in generic_kernel_paths]):
        download_generic_kernels()
    for kernel in generic_kernel_paths:
        spice.furnsh(str(kernel))

In [115]:
spice.kclear()

In [116]:
load_generic_kernels()

In [117]:
#| export

def show_loaded_kernels():
    "Print overview of loaded kernels."
    count = spice.ktotal("all")
    if count == 0:
        print("No kernels loaded at this time.")
    else:
        print("The loaded files are:\n(paths relative to kernels.KERNEL_STORAGE)\n")
    for which in range(count):
        out = spice.kdata(which, "all", 100, 100, 100)
        print("Position:", which)
        p = Path(out[0])
        print("Path", p.relative_to(KERNEL_STORAGE))
        print("Type:", out[1])
        print("Source:", out[2])
        print("Handle:", out[3])
        # print("Found:", out[4])

In [118]:
show_loaded_kernels()

The loaded files are:
(paths relative to kernels.KERNEL_STORAGE)

Position: 0
Path generic/lsk/naif0012.tls
Type: TEXT
Source: 
Handle: 0
Position: 1
Path generic/pck/pck00010.tpc
Type: TEXT
Source: 
Handle: 0
Position: 2
Path generic/pck/de-403-masses.tpc
Type: TEXT
Source: 
Handle: 0
Position: 3
Path generic/spk/planets/de430.bsp
Type: SPK
Source: 
Handle: 13
Position: 4
Path generic/spk/satellites/mar097.bsp
Type: SPK
Source: 
Handle: 14
